# Agents

- Agents are one of the newest (and most experimental) parts of Langchain, but with huge potential for LLM Applications
- They combine what we already learned about Model IO, data connections, and chains, but allow us to create more robust applications
- Agents take this one step further with the ReACT framework

At their core, agents allow LLM to connect to tools (e.g. Wikipedia, calculator, google search, etc.) and conduct a structured approach to complete a task based on ReACT: Reasoning and Acting.

ReACT prompting can help an LLM reason about a task and perform correct actions based on observations.

The agent is given a task and can reason which tools are appropriate to use and can then utilize those results to continue on via an internal chain until it solves the task.

Internally, LangChain labels the thinking in terms of:
- Observation
- Thought
- Action

**Why are they useful?**
- Extremely powerful, especially when we combine them with custom tools
- Imagine an agent with access to internal corporate documents and the ability to conduct outside relevant searches, you have a very powerful corporate assistant with perfect recall and memory to answer questions


**Attention!:** They are not perfect and we may experience funny behaviors as we experiment with them.

# Agent basics

In [1]:
from langchain.agents import load_tools, initialize_agent, AgentType
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI

How to check available tools? https://python.langchain.com/docs/modules/agents/tools/

In [3]:
# What tools should the agent have access to?
llm = ChatOpenAI(temperature=0)  # usually, with agents, we want to keep hallucinations very low
tools = load_tools(["llm-math"], llm=llm)

In [10]:
# Available attributes in agent type
# Most common will be 'ZERO_SHOT_REACT_DESCRIPTION'
# In this case, we will give 0 examples to the model
dir(AgentType)

['CHAT_CONVERSATIONAL_REACT_DESCRIPTION',
 'CHAT_ZERO_SHOT_REACT_DESCRIPTION',
 'CONVERSATIONAL_REACT_DESCRIPTION',
 'OPENAI_FUNCTIONS',
 'OPENAI_MULTI_FUNCTIONS',
 'REACT_DOCSTORE',
 'SELF_ASK_WITH_SEARCH',
 'STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION',
 'ZERO_SHOT_REACT_DESCRIPTION',
 '__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getnewargs__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__members__',
 '__mod__',
 '__module__',
 '__mul__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmod__',
 '__rmul__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'capitalize',
 'casefold',
 'center',
 'count',
 'encode',
 'endswith',
 'expandtabs',
 'find',
 'format',
 'format_map',
 'index',
 'isalnum',
 'isalpha',
 'isascii',


In [14]:
agent = initialize_agent(tools=tools, llm=llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [15]:
# Agent is just a chain
agent.run("What is 134292 times 29384920?")



> Entering new AgentExecutor chain...
I need to multiply 134292 by 29384920.
Action: Calculator
Action Input: 134292 * 29384920
Observation: Answer: 3946159676640
Thought:I now know the final answer
Final Answer: 3946159676640

> Finished chain.


'3946159676640'

In [16]:
agent = initialize_agent(tools=tools, llm=llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION)
agent.run("What is 134292 times 29384920?")

'3946159676640'

In [17]:
134292*29384920

3946159676640

# Agent Tools
Integrations: https://python.langchain.com/docs/integrations/tools/

Usually, requires 2 things:
- Install a package
- Get a key

We'll use as an example, SerpAPI (free 100 searches per month): https://serpapi.com/dashboard
Requirements:
- install `google-search-results`
- Set `SERPAPI_API_KEY` environment key

In [2]:
llm = ChatOpenAI(temperature=0)
tools = load_tools(["serpapi", "llm-math"], llm=llm)

In [3]:
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [4]:
agent.run("What year was Albert Einstein born? What is that year multiplied by 5?")



> Entering new AgentExecutor chain...
I can use the search tool to find the year Albert Einstein was born. Then I can use the calculator to multiply that year by 5.
Action: Search
Action Input: "Albert Einstein birth year"
Observation: March 14, 1879
Thought:Now I can use the calculator to multiply 1879 by 5.
Action: Calculator
Action Input: 1879 * 5
Observation: Answer: 9395
Thought:I now know the final answer
Final Answer: Albert Einstein was born in 1879. When multiplied by 5, that is equal to 9395.

> Finished chain.


'Albert Einstein was born in 1879. When multiplied by 5, that is equal to 9395.'

In [7]:
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.agents.agent_toolkits import create_python_agent

In [8]:
llm = ChatOpenAI(temperature=0)
agent = create_python_agent(
    tool=PythonREPLTool(),
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [11]:
python_list = [1, 3, 2, 4, 1, 2, 3, 4, 1, 1, 2, 3, 5, 10]

In [12]:
agent.run(f"""Sort this Python list: {python_list}""")



> Entering new AgentExecutor chain...


Python REPL can execute arbitrary code. Use with caution.


I can use the `sorted()` function to sort the list in ascending order.
Action: Python_REPL
Action Input: sorted([1, 3, 2, 4, 1, 2, 3, 4, 1, 1, 2, 3, 5, 10])
Observation: 
Thought:The list is sorted in ascending order.
Final Answer: [1, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 5, 10]

> Finished chain.


'[1, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 5, 10]'

# Custom tools

In [14]:
from langchain.agents import tool  # this is a decorator

In [16]:
# LLM -> Tools docstrings -> pick tool -> action -> observation -> 

@tool
def coolest_guy(text: str) -> str:
    '''Returns the name of the coolest person in the universe. 
    Expects an input of an empty string '' and returns the 
    name of the coolest person in the universe'''
    return "James Bond"

### Without custom tool

In [18]:
llm = ChatOpenAI(temperature=0)
tools = load_tools(["wikipedia", "llm-math"], llm=llm)

In [19]:
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [20]:
agent.run("Who is the coolest person in the universe?")



> Entering new AgentExecutor chain...
This question is subjective and cannot be answered with factual information.
Final Answer: There is no definitive answer to this question as it is based on personal opinion.

> Finished chain.


'There is no definitive answer to this question as it is based on personal opinion.'

### With custom tool

In [22]:
llm = ChatOpenAI(temperature=0)

tools = load_tools(["wikipedia", "llm-math"], llm=llm)
tools = tools + [coolest_guy]

In [23]:
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [24]:
agent.run("Who is the coolest person in the universe?")



> Entering new AgentExecutor chain...
I can use the coolest_guy function to find the answer.
Action: coolest_guy
Action Input: ''
Observation: James Bond
Thought:I now know the final answer
Final Answer: James Bond

> Finished chain.


'James Bond'

In [25]:
# Example for internal company api
# @tool
# def some_api(text: str) -> str:
#     '''Connect this API and use this key...'''
#     return answer

In [26]:
# This error is a flaw of agents, ideally, it would just say it can't get the answer, and not raise an error
agent.run("What time is it right now?")



> Entering new AgentExecutor chain...


OutputParserException: Could not parse LLM output: `I don't have access to real-time information, so I can't answer this question directly.
Action: N/A`

In [27]:
from datetime import datetime

# This may be a weird python docstring, but it's the best format for the LLM to understand

@tool
def get_current_time(text: str) -> str:
    '''Returns the current time. Use this for any questions regarding the current time.
    Input is an empty string and the current time is returned in a string format.
    Only use this function for the current time, other time related questions should use another tool.'''
    return str(datetime.now())

In [28]:
tools = tools + [get_current_time]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [29]:
agent.run("What time is it right now?")



> Entering new AgentExecutor chain...
I should use the get_current_time tool to get the current time.
Action: get_current_time
Action Input: ''
Observation: 2023-09-29 12:35:03.931386
Thought:I now know the current time.
Final Answer: The current time is 2023-09-29 12:35:03.931386.

> Finished chain.


'The current time is 2023-09-29 12:35:03.931386.'

In [30]:
agent.run("What time did Pearl Harbor attack happen?")



> Entering new AgentExecutor chain...
I should use Wikipedia to find the answer to this historical event.
Action: Wikipedia
Action Input: "Pearl Harbor attack"
Observation: Page: Attack on Pearl Harbor
Summary: The attack on Pearl Harbor was a surprise military strike by the Imperial Japanese Navy Air Service upon the United States against the U.S. naval base at Pearl Harbor in Honolulu, Hawaii, just before 8:00 a.m. (local time) on Sunday, December 7, 1941. The United States was a neutral country at the time; the attack led to its formal entry into World War II on the side of the Allies the next day. The Japanese military leadership referred to the attack as the Hawaii Operation and Operation AI, and as Operation Z during its planning.The attack was preceded by months of negotiations between the U.S. and Japan over the future of the Pacific. Japanese demands included that the U.S. end its sanctions against Japan, cease aiding China in the Second Sino-Japanese war, and allow Japan to

'The attack on Pearl Harbor happened just before 8:00 a.m. on Sunday, December 7, 1941.'

# Conversation Agents

In [31]:
from langchain.memory import ConversationBufferMemory

In [52]:
memory = ConversationBufferMemory(memory_key="chat_history")

In [57]:
llm = ChatOpenAI(temperature=0)

In [58]:
tools = load_tools(["llm-math"], llm=llm)

In [59]:
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    verbose=True
)

In [60]:
agent.run(input="What are some good Thai food recipes?")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: Thai cuisine is known for its bold flavors and aromatic ingredients. Here are a few popular Thai food recipes that you might enjoy:

1. Pad Thai: This stir-fried noodle dish is a classic Thai favorite. It typically includes rice noodles, shrimp or chicken, eggs, tofu, bean sprouts, and a tangy sauce made from tamarind paste, fish sauce, and lime juice.

2. Green Curry: This spicy curry is made with a paste of green chilies, garlic, lemongrass, and other herbs and spices. It is typically cooked with coconut milk, vegetables, and your choice of meat or tofu.

3. Tom Yum Soup: This hot and sour soup is a popular Thai appetizer. It is made with a flavorful broth, shrimp or chicken, mushrooms, lemongrass, lime juice, and chili peppers.

4. Massaman Curry: This rich and flavorful curry is made with a paste of roasted spices, including cinnamon, cardamom, and cloves. It is typically cooked with coconut milk, pota

"Thai cuisine is known for its bold flavors and aromatic ingredients. Here are a few popular Thai food recipes that you might enjoy:\n\n1. Pad Thai: This stir-fried noodle dish is a classic Thai favorite. It typically includes rice noodles, shrimp or chicken, eggs, tofu, bean sprouts, and a tangy sauce made from tamarind paste, fish sauce, and lime juice.\n\n2. Green Curry: This spicy curry is made with a paste of green chilies, garlic, lemongrass, and other herbs and spices. It is typically cooked with coconut milk, vegetables, and your choice of meat or tofu.\n\n3. Tom Yum Soup: This hot and sour soup is a popular Thai appetizer. It is made with a flavorful broth, shrimp or chicken, mushrooms, lemongrass, lime juice, and chili peppers.\n\n4. Massaman Curry: This rich and flavorful curry is made with a paste of roasted spices, including cinnamon, cardamom, and cloves. It is typically cooked with coconut milk, potatoes, onions, and your choice of meat or tofu.\n\n5. Mango Sticky Rice: 

In [61]:
agent.run(input="Which one of those dishes tends to be the spiceiest?")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: Among the dishes mentioned, the Green Curry is typically the spiciest. It is made with a paste of green chilies, which gives it its vibrant color and fiery heat. However, the level of spiciness can vary depending on personal preference and the amount of chili used in the recipe. If you prefer milder flavors, you can always adjust the amount of chili or use a milder variety of chili.

> Finished chain.


'Among the dishes mentioned, the Green Curry is typically the spiciest. It is made with a paste of green chilies, which gives it its vibrant color and fiery heat. However, the level of spiciness can vary depending on personal preference and the amount of chili used in the recipe. If you prefer milder flavors, you can always adjust the amount of chili or use a milder variety of chili.'

In [62]:
agent.run(input="Give me a grocery shopping list for that dish.")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: Sure! Here's a grocery shopping list for Green Curry:

- Green curry paste
- Coconut milk
- Chicken, beef, shrimp, or tofu (depending on your preference)
- Vegetables (such as bell peppers, eggplant, and bamboo shoots)
- Thai basil leaves
- Fish sauce
- Palm sugar or brown sugar
- Kaffir lime leaves (optional)
- Thai bird's eye chilies (optional, for extra spiciness)
- Jasmine rice (to serve with the curry)

These are the basic ingredients you'll need to make a delicious Green Curry. Of course, you can customize the recipe by adding or omitting ingredients based on your taste preferences. Enjoy cooking! Let me know if there's anything else I can assist you with.

> Finished chain.


"Sure! Here's a grocery shopping list for Green Curry:\n\n- Green curry paste\n- Coconut milk\n- Chicken, beef, shrimp, or tofu (depending on your preference)\n- Vegetables (such as bell peppers, eggplant, and bamboo shoots)\n- Thai basil leaves\n- Fish sauce\n- Palm sugar or brown sugar\n- Kaffir lime leaves (optional)\n- Thai bird's eye chilies (optional, for extra spiciness)\n- Jasmine rice (to serve with the curry)\n\nThese are the basic ingredients you'll need to make a delicious Green Curry. Of course, you can customize the recipe by adding or omitting ingredients based on your taste preferences. Enjoy cooking! Let me know if there's anything else I can assist you with."